In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
import numpy as np
np.set_printoptions(threshold=np.inf)
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans,DBSCAN
from sklearn.cluster import AgglomerativeClustering as ac
from sklearn.linear_model import LogisticRegression as LR
from sklearn.model import train_test_split as tts
from sklearn.metrics import accuracy_score as acc
from sklearn import SVC
from sklearn import metrics
from sklearn.metrics import  plot_roc_curve
from sklearn.model_selection import  train_test_split

In [35]:
# ——————————读取数据——————————#
# 航班数据
flight_data_2009 = pd.read_csv('./source/2009.csv',encoding='unicode_escape')
# flight_data_2010 = pd.read_csv('./source/2010.csv',encoding='unicode_escape')
# flight_data_09_10 = pd.concat(
   # [flight_data_2009,flight_data_2010]).reset_index(drop=True)
#flight_data_2011 = pd.read_csv('./source/2011.csv',encoding='unicode_escape')
#flight_data_2012 = pd.read_csv('./source/2012.csv',encoding='unicode_escape')
#flight_data_2013 = pd.read_csv('./source/2013.csv',encoding='unicode_escape')
#flight_data_09_13 = pd.concat(
#    [flight_data_2009,flight_data_2010,flight_data_2011,flight_data_2012,flight_data_2013]).reset_index(drop=True)

#flight_data_2014 = pd.read_csv('./source/2014.csv',encoding='unicode_escape')
#flight_data_2015 = pd.read_csv('./source/2015.csv',encoding='unicode_escape')
#flight_data_2016 = pd.read_csv('./source/2016.csv',encoding='unicode_escape')
#flight_data_2017 = pd.read_csv('./source/2017.csv',encoding='unicode_escape')
#flight_data_2018 = pd.read_csv('./source/2018.csv',encoding='unicode_escape')

In [36]:
# 保存数据集
flight_data_2009.to_csv('./clear_data/temp_train_data.csv',index=False)
flight_data_2009

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,2009-01-01,XE,1204,DCA,EWR,1100,1058.0,-2.0,18.0,1116.0,...,62.0,68.0,42.0,199.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2009-01-01,XE,1206,EWR,IAD,1510,1509.0,-1.0,28.0,1537.0,...,82.0,75.0,43.0,213.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2009-01-01,XE,1207,EWR,DCA,1100,1059.0,-1.0,20.0,1119.0,...,70.0,62.0,36.0,199.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2009-01-01,XE,1208,DCA,EWR,1240,1249.0,9.0,10.0,1259.0,...,77.0,56.0,37.0,199.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2009-01-01,XE,1209,IAD,EWR,1715,1705.0,-10.0,24.0,1729.0,...,105.0,77.0,40.0,213.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6429333,2009-12-31,YV,2846,YUM,PHX,615,605.0,-10.0,19.0,624.0,...,67.0,71.0,39.0,160.0,NaN,NaN,NaN,NaN,NaN,NaN
6429334,2009-12-31,YV,2850,YUM,PHX,1929,1919.0,-10.0,7.0,1926.0,...,70.0,61.0,48.0,160.0,NaN,NaN,NaN,NaN,NaN,NaN
6429335,2009-12-31,YV,2866,YUM,PHX,1400,1400.0,0.0,29.0,1429.0,...,55.0,70.0,32.0,160.0,0.0,0.0,15.0,0.0,0.0,NaN
6429336,2009-12-31,YV,2868,YUM,PHX,1620,1615.0,-5.0,10.0,1625.0,...,67.0,61.0,45.0,160.0,NaN,NaN,NaN,NaN,NaN,NaN


数据集列的含义
FL_DATE 航班日期，yy/mm/dd//
OP_CARRIER 航空公司标识符//
OP_CARRIER_FL_NUM 航班号//
ORIGIN 起始机场代码//
DEST 目的地机场代码//
CRS_DEP_TIME 计划出发时间//
DEP_TIME 实际出发时间//
DEP_DELAY 出发总延误分钟//
TAXI_OUT 从始发机场登机口出发到飞机起飞之间经过的时间，起飞滑行时间
WHEELS_OFF 飞机轮子离开地面的时间点
WHEELS_ON 飞机轮子接触地面的时间点
TAXI_IN 从机轮启动到到达目的地机场登机口所经过的时间，降落滑行时间
CRS_ARR_TIME 计划到达时间//
ARR_TIME 实际到达时间//
ARR_DELAY 到达总延误分钟//
CANCELLED 航班取消（1 = 取消）//
CANCELLATION_CODE 取消代码 A - 航空公司/承运人；B - 天气；C - 国家航空系统；D - 安全
DIVERTED 飞机降落在不合时宜的机场
CRS_ELAPSED_TIME 飞行旅行所需的计划时间量//
ACTUAL_ELAPSED_TIME AIR_TIME+TAXI_IN+TAXI_OUT
AIR_TIME wheels_off 和 wheels_on 时间之间的持续时间
DISTANCE 两个机场之间的距离
CARRIER_DELAY 几分钟内航空公司造成的延误
WEATHER_DELAY 天气原因延误//
NAS_DELAY 空气系统造成的延迟
SECURITY_DELAY 安全问题造成的延误
LATE_AIRCRAFT_DELAY 安检造成的延误

1.计划飞行时间（单位：分钟）【间接反映航线距离】
2.计划起飞日期&计划到达日期【用以匹配天气信息】yyyy-mm-dd
3.起飞延误时间（单位：分钟）负数为提前，正数为延误
4.到达延误时间（单位：分钟）
5.天气用机场和时间去连接

In [12]:
# 读取训练集
flight_data = pd.read_csv('./clear_data/temp_train_data.csv')

# 重命名列
flight_data['航班是否取消'] = flight_data['CANCELLED']

flight_data['航班日期'] = flight_data['FL_DATE']
flight_data['航空公司'] = flight_data['OP_CARRIER']
flight_data['航班号'] = flight_data['OP_CARRIER_FL_NUM']
flight_data['出发机场'] = flight_data['ORIGIN']
flight_data['到达机场'] = flight_data['DEST']

flight_data['计划起飞时间'] = flight_data['CRS_DEP_TIME']
flight_data['实际起飞时间'] = flight_data['DEP_TIME']
flight_data['出发延误总分钟'] = flight_data['DEP_DELAY']

flight_data['计划到达时间'] = flight_data['CRS_ARR_TIME']
flight_data['实际到达时间'] = flight_data['ARR_TIME']
flight_data['到达延误总分钟'] = flight_data['ARR_DELAY']

# 计划飞行时间
flight_data['计划飞行时间'] = flight_data['计划到达时间'] - flight_data['计划起飞时间']

In [13]:
# 删除用不到的字段
del flight_data['FL_DATE']
del flight_data['OP_CARRIER']
del flight_data['CANCELLED']
del flight_data['OP_CARRIER_FL_NUM']
del flight_data['ORIGIN']
del flight_data['DEST']
del flight_data['CRS_DEP_TIME']
del flight_data['DEP_TIME']
del flight_data['DEP_DELAY']
del flight_data['CRS_ARR_TIME']
del flight_data['ARR_TIME']
del flight_data['ARR_DELAY']
del flight_data['LATE_AIRCRAFT_DELAY']
del flight_data['SECURITY_DELAY']
del flight_data['NAS_DELAY']
del flight_data['CARRIER_DELAY']
del flight_data['DISTANCE']
del flight_data['AIR_TIME']
del flight_data['ACTUAL_ELAPSED_TIME']
del flight_data['DIVERTED']
del flight_data['CANCELLATION_CODE']
del flight_data['TAXI_IN']
del flight_data['WHEELS_ON']
del flight_data['WHEELS_OFF']
del flight_data['TAXI_OUT']
del flight_data['Unnamed: 27']
del flight_data['CRS_ELAPSED_TIME']
del flight_data['WEATHER_DELAY']

In [14]:
# 实际到达时间超过计划到达时间15min或者取消航班作为延误的判断指标，给数据打上标签
def isDelay(t,c):
    if(t >= 15 or c==1): return 1
    else: return 0

flight_data['航班是否延误'] = flight_data.apply(
    lambda row: isDelay(row['到达延误总分钟'],row['航班是否取消']),axis=1)

In [15]:
# 判断正负样本的分布
flight_data['航班是否延误'].value_counts()

0    5171799
1    1257539
Name: 航班是否延误, dtype: int64

In [18]:
# 看看是否有缺失值
flight_data.isnull().any()

航班是否取消     False
航班日期       False
航空公司       False
航班号        False
出发机场       False
到达机场       False
计划起飞时间     False
实际起飞时间      True
出发延误总分钟     True
计划到达时间     False
实际到达时间      True
到达延误总分钟     True
计划飞行时间     False
航班是否延误     False
dtype: bool

In [ ]:
# 缺失值处理
flight_data['实际起飞时间'] = flight_data['实际起飞时间'].fillna(flight_data['实际起飞时间'].mean()) #均值填充
flight_data['出发延误总分钟'] = flight_data['出发延误总分钟'].fillna(flight_data['出发延误总分钟'].mean()) #均值填充
flight_data['实际到达时间'] = flight_data['实际到达时间'].fillna(flight_data['实际到达时间'].mean()) #均值填充
flight_data['到达延误总分钟'] = flight_data['到达延误总分钟'].fillna(flight_data['到达延误总分钟'].mean()) #均值填充
flight_data['出发延误总分钟'] = flight_data['出发延误总分钟'].fillna(flight_data['出发延误总分钟'].mean()) #均值填充

In [ ]:
# 读取机场和对应城市
airport_city = pd.read('./source/us_airports.csv',encoding='unicode_escape',low_memory=False)

In [ ]:
# 处理缺失值
airport_city['iata_code'] = airport_city['iata_code'].fillna(
    airport_city['iata_code'].value_counts().index[0]) #众数填充
airport_city['iata_code'] = airport_city['iata_code'].fillna(
    airport_city['iata_code'].value_counts().index[0]) #众数填充

In [ ]:
# 拼接机场和对应城市
airport_city['机场编号'] = airport_city['iata_code']
del airport_city['iata_code']

In [ ]:
# 抽取随机样本


In [ ]:
# 做ROC图像
# plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
# plt.rcParams['axes.unicode_minus'] = False;
#
# fig = plt.figure(figsize=(8,8),dpi = 100,facecolor='white',edgecolor='black')
# ax.fig.add_subplot(1,1,1)
# ax.plot(fpr_CART_Class,c='red')
# ax.plot(fpr_lr,tpr_lr,c='orange')
# plt.legend(('决策树','logistic回归'),loc = 'best')
# plt.show(fig)
# fig.savefig('./figure/roc.jpg')
# plt.close(fig)